In [ ]:
import os
from datetime import time, date, datetime, timedelta
from IPython.display import display, HTML
import polars as pl

os.environ["POLARS_FMT_MAX_ROWS"] = "50"
os.environ["POLARS_FMT_MAX_COLS"] = "50"
os.environ["POLARS_FMT_COL_MAX_WIDTH"] = "50"
pl.Config(fmt_str_lengths=50)

def print_df(df: pl.DataFrame, title=None):
	if title:
		display(HTML(f"<h2>{title}</h2>"))
	display(HTML(df._repr_html_()))

now = datetime.now()

# import httpx
# from src.user_list import UserList
# user_list = UserList()
# async with httpx.AsyncClient() as client:
# 	user_list = await UserList.from_web("cosmicwaves", client)
# 	user_list.write_parquet("data/user_list.parquet")

user_list = pl.read_parquet("data/user_list.parquet")
anime_db = pl.read_parquet("data/anime.parquet")

common_cols = set(user_list.columns).intersection(set(anime_db.columns))
if len(common_cols) > 1:
	raise Exception(f"Common columns found between user_list and anime_db: {common_cols}")

user_animes = user_list.join(anime_db, on="anime_id", how="inner", validate="1:1")

if user_animes.height < user_list.height:
	print(f"Warning: {user_list.height - user_animes.height} anime not found in database")

print_df(user_animes.head(10), "User Animes")

In [ ]:
# TODO build every lazy frame needed and then pl.gather

schedule = user_animes.filter((pl.col("user_status") == "Watching") & pl.col("air_now")).select(
	"title",
	"broadcast_day",
	"broadcast_time",
	"broadcast_timezone",
)
print_df(schedule, "Schedule")
WEEK_DAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
import pytz
# TODO take user timezone into account
def get_dt(air_day: str, air_time: time, air_tz_str: str, user_tz_str: str):
	"Converts the given day and time to user local time."
	
	user_tz = pytz.timezone(user_tz_str)
	air_tz = pytz.timezone(air_tz_str)

	# Get start of the week
	now = datetime.now(air_tz)
	start_of_week = now.date() - timedelta(days=now.weekday())

	# Get the day and time
	# TODO Remove 's' from the end of the day to make it singular
	day_num = WEEK_DAYS.index(air_day)
	air_at = datetime.combine(start_of_week + timedelta(days=day_num), air_time)

	# Localize the datetime to the JST timezone
	air_at = air_tz.localize(air_at)

	# Normalize the datetime to handle daylight saving time transitions
	air_at = air_tz.normalize(air_at)

	# Convert the datetime to the local timezone
	return air_at.astimezone(user_tz)


next_releases = user_animes.filter((pl.col("user_status") == "Plan to Watch") & ~pl.col("air_started") & ~pl.col("air_ended")).sort("air_start", nulls_last=True).select(
	"title",
	# pl.col("air_start").dt.to_string("%A %d %B %Y").alias("air_start"),
	"air_start",
	# pl.col("broadcast_time").dt.to_string("at %H:%M").alias("broadcast_time"),
	"broadcast_time",
	"broadcast_timezone",
)
print_df(next_releases, "Next releases")